# IBM watsonx.ai + Langflow Workshop

This notebook walks you through:

- Installing and configuring **Langflow** for use with **IBM watsonx.ai**
- Setting up watsonx.ai credentials for use in Langflow
- Building a **basic chat flow** in Langflow that uses the IBM watsonx.ai component
- Calling your Langflow + watsonx flow from Python via the **Langflow API**

> ⚠️ This notebook does **not** replace the Langflow UI. You still build the flow
> visually in Langflow, but this notebook helps with environment setup and API usage.


## 0. Prerequisites

To run this workshop you need:

- Python 3.10+
- An IBM Cloud account with access to **watsonx.ai**
- A watsonx.ai **service instance**, **project**, and **API key**
- A running **Langflow** instance (Desktop, Docker, or `langflow run`)
- Basic familiarity with Python, terminals, and web UIs

Useful links (open in your browser):

- Langflow docs: `https://docs.langflow.org/`
- IBM bundle docs: `https://docs.langflow.org/bundles-ibm`


## 1. Install Python dependencies

You only need to run this cell **once per environment**.

If your Jupyter environment supports IPython magics, you can keep `%pip`.
Otherwise, remove the leading `%` and run as plain `pip`.


In [ ]:
# Install dependencies (run once)
# Uncomment the next line if your environment supports IPython magics:
# %pip install -U langflow "langflow[all]" langchain-ibm ibm-watsonx-ai python-dotenv requests

print("If needed, install with: pip install -U langflow 'langflow[all]' langchain-ibm ibm-watsonx-ai python-dotenv requests")

## 2. Configure IBM watsonx.ai credentials

You’ll need three pieces of information from IBM Cloud:

- **IBM Cloud API key** for watsonx.ai
- **Service URL** (depends on your region), for example:
  - `https://us-south.ml.cloud.ibm.com` (Dallas)
  - `https://eu-de.ml.cloud.ibm.com` (Frankfurt)
  - `https://eu-gb.ml.cloud.ibm.com` (London)
  - `https://jp-tok.ml.cloud.ibm.com` (Tokyo)
  - `https://au-syd.ml.cloud.ibm.com` (Sydney)
- **Project ID** for your watsonx.ai project

This cell will:

- Prompt you for your API key, URL, and project ID
- Store them in environment variables:
  - `WATSONX_APIKEY` and `WATSONX_API_KEY` (both set for compatibility)
  - `WATSONX_URL`
  - `WATSONX_PROJECT_ID`
- Enable the Langflow feature that lets global variables fall back to env vars:
  - `LANGFLOW_FALLBACK_TO_ENV_VAR=True`


In [ ]:
import os
from getpass import getpass

print("👉 Enter your IBM watsonx.ai credentials (they stay only in this notebook session).\n")


WATSONX_API_KEY = getpass("IBM Cloud API key for watsonx.ai: ")
WATSONX_URL = input("watsonx.ai URL (e.g. https://us-south.ml.cloud.ibm.com): ").strip()
WATSONX_PROJECT_ID = input("watsonx.ai Project ID: ").strip()

# Store in environment variables (some libraries expect one name, some the other)
os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY
os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY
os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

# Optional but useful for Langflow global variables
os.environ["LANGFLOW_FALLBACK_TO_ENV_VAR"] = "True"

print("\n✅ Environment variables set: WATSONX_APIKEY, WATSONX_API_KEY, WATSONX_URL, WATSONX_PROJECT_ID, LANGFLOW_FALLBACK_TO_ENV_VAR")

## 3. Quick sanity check: Chat with a watsonx.ai model

Before involving Langflow, make sure your watsonx credentials and network work
by calling a watsonx.ai chat model directly using `langchain-ibm`.


In [ ]:
from langchain_ibm import ChatWatsonx

# Pick any chat-capable model that is enabled in your region.
# You can adjust this later.
WATSONX_MODEL_ID = "ibm/granite-3-8b-instruct"

parameters = {
    "temperature": 0.2,
    "max_tokens": 256,
}

chat = ChatWatsonx(
    model_id=WATSONX_MODEL_ID,
    url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    params=parameters,
)

messages = [
    ("system", "You are a concise assistant for IBM watsonx.ai + Langflow workshops."),
    ("human", "In 3 bullet points, explain what Langflow is."),
]

print("⏳ Calling watsonx.ai...\n")
response = chat.invoke(messages)
print("=== watsonx.ai response ===\n")
print(response.content)

## 4. Start Langflow

You can run Langflow in several ways:

- **Langflow Desktop** (recommended for beginners)
- **Docker**: `docker run -p 7860:7860 langflowai/langflow:latest`
- **CLI** in a virtual environment: `langflow run`

For this workshop, we assume Langflow is running locally on:

- `http://localhost:7860` (default for `langflow run`)
- Or `http://localhost:7861` if you're using watsonx Orchestrate Developer Edition with `--with-langflow`

> 🧠 Tip: Start Langflow **after** setting your watsonx environment variables so
> components and global variables can see them.


In [ ]:
# Example: start Langflow from a terminal (not directly from this notebook)
# (Run these commands in a separate shell, not inside Python)

example_commands = '''
# In your terminal (NOT in Python):

# 1) Activate your virtual environment (if you created one)
#    e.g. on macOS/Linux:
#    source .venv/bin/activate
#
# 2) Make sure environment variables are set (API key, URL, project ID)
#    export WATSONX_APIKEY=...
#    export WATSONX_URL=...
#    export WATSONX_PROJECT_ID=...
#
# 3) Start Langflow:
#    langflow run
#
# 4) Open the UI in your browser:
#    http://localhost:7860
'''

print(example_commands)

## 5. Build a basic watsonx.ai chat flow in Langflow

Now switch to the **Langflow UI** in your browser and build a simple flow that
uses the IBM bundle.

### 5.1 Create a new flow

1. In the Langflow UI, click **New Flow**.
2. Name it something like: `watsonx_basic_chat`.
3. Optionally add a description: `Simple chat using IBM watsonx.ai`.

### 5.2 Add components

Drag the following components onto the canvas:

1. **Chat Input** (category: *Input / Output*)
2. **Prompt Template** (category: *Processing*)
3. **IBM watsonx.ai** (category: *Bundles → IBM*)
4. **Chat Output** (category: *Input / Output*)

Connect them like this:

`Chat Input → Prompt Template → IBM watsonx.ai → Chat Output`

### 5.3 Configure the Prompt Template

Click the **Prompt Template** component and set:

- **Template** (example):

```text
You are an AI assistant for an IBM watsonx.ai + Langflow workshop.
Answer clearly and keep responses under 8 sentences.

User question:
{input}
```

- Make sure the **input variable** name (e.g. `input`) matches the incoming
  field from the Chat Input component. The default is usually fine.

### 5.4 Configure the IBM watsonx.ai component

Click the **IBM watsonx.ai** component and set:

- **url**: `{{WATSONX_URL}}` or your region URL directly
- **project_id**: `{{WATSONX_PROJECT_ID}}` or your project ID directly
- **api_key**: `{{WATSONX_APIKEY}}` (or another global variable name)

If you use `{{...}}` syntax, make sure you have matching **Global Variables**
defined in **Settings → Global Variables** in Langflow, or that
`LANGFLOW_FALLBACK_TO_ENV_VAR=True` and the environment variables are set.

For **model_name**:

- Once `url`, `project_id`, and `api_key` are valid, the component can
  auto-populate available models from your watsonx.ai deployment.
- Choose a chat-capable model, for example:
  - `ibm/granite-3-8b-instruct`

You can adjust other parameters as needed, for example:

- `temperature = 0.2`
- `max_tokens = 512`
- `top_p = 0.9`

### 5.5 Test the flow in the Langflow UI

1. Click the **Chat Input** component.
2. Enter a message, such as: `Explain what this flow does.`
3. Click **Run** or use the chat panel.
4. You should see a response appear in **Chat Output**, generated by your
   watsonx.ai model through Langflow.


## 6. Export your flow as JSON (optional)

To make this flow portable or use it with other tools (like watsonx Orchestrate):

1. In the Langflow UI, open your `watsonx_basic_chat` flow.
2. Click **Share → Export**.
3. Choose **Export as JSON** (optionally check “Save with my API keys” if you
   need the credentials embedded for local testing).
4. Save the JSON file (for example, `watsonx_basic_chat.json`).

You can re-import this JSON into any Langflow instance to recreate the flow.


## 7. Call your Langflow + watsonx flow from Python

Every Langflow flow can be executed from code using the **Langflow API**.

We’ll prepare a small helper in Python to call your flow via HTTP.

You need:

- Your **Langflow server URL**, e.g. `http://localhost:7860`
- A **Langflow API key** (see `Settings → Langflow API Keys` in the UI, or the
  docs on API keys and authentication)
- Your flow’s **ID** (from `Share → API access` or from the flow URL)


In [ ]:
import uuid
import requests

print("👉 Configure Langflow API access.\n")


LANGFLOW_SERVER_URL = input("Langflow server URL (e.g. http://localhost:7860): ").strip()
FLOW_ID = input("Flow ID (from Langflow 'Share → API access'): ").strip()
LANGFLOW_API_KEY = getpass("Langflow API key (Settings → Langflow API Keys): ")

# Optional: store in environment variables for reuse
os.environ["LANGFLOW_SERVER_URL"] = LANGFLOW_SERVER_URL
os.environ["LANGFLOW_FLOW_ID"] = FLOW_ID
os.environ["LANGFLOW_API_KEY"] = LANGFLOW_API_KEY

print("\n✅ Langflow API configuration saved in this session.")

In [ ]:
def run_langflow_chat(message: str, session_id: str | None = None) -> dict:
    """Call the Langflow flow via /api/v1/run/{flow_id} and return the JSON response."""
    if session_id is None:
        session_id = f"chat-{uuid.uuid4()}"

    url = f"{os.environ['LANGFLOW_SERVER_URL'].rstrip('/')}/api/v1/run/{os.environ['LANGFLOW_FLOW_ID']}"

    headers = {
        "Content-Type": "application/json",
        "x-api-key": os.environ["LANGFLOW_API_KEY"],
    }

    payload = {
        # basic chat-style request; Langflow examples use this pattern
        "input_value": message,
        "session_id": session_id,
        "input_type": "chat",
        "output_type": "chat",
        "output_component": "",  # default first chat output
        "tweaks": None,           # no runtime overrides yet
    }

    response = requests.post(url, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()


print("Helper function `run_langflow_chat` is ready.")

In [ ]:
# Test: run your watsonx + Langflow flow once from Python

test_message = "Give me 3 workshop ideas for using IBM watsonx.ai with Langflow."
print("⏳ Calling Langflow flow...\n")
result = run_langflow_chat(test_message)

print("=== Raw API response keys ===")
print(result.keys())

# Try to extract the first chat output text, if present
try:
    outputs = result.get("outputs", [])
    if outputs:
        # outputs[0]["outputs"][0]["results"]["message"]["text"]
        first_output = outputs[0]["outputs"][0]["results"]["message"]["text"]
        print("\n=== First chat output ===\n")
        print(first_output)
    else:
        print("No 'outputs' field found in response. Inspect `result` manually.")
except Exception as e:
    print("Could not parse chat output from response. Inspect `result` directly.")
    print("Error:", e)

## 8. Advanced: tweak IBM watsonx.ai parameters at runtime

Langflow supports **tweaks**: one-time overrides of component parameters when you
call a flow via the API.

To use tweaks:

1. In Langflow, find the **IBM watsonx.ai** component ID, something like:
   - `IBMwatsonxModel-XXXXXX`
   You can see this in the API access modal or in the flow JSON.
2. Use that ID as a key in the `tweaks` dictionary.

Example (Python): override the model and temperature for a single request.


In [ ]:
def run_langflow_chat_with_tweaks(message: str, tweaks: dict, session_id: str | None = None) -> dict:
    if session_id is None:
        session_id = f"chat-{uuid.uuid4()}"

    url = f"{os.environ['LANGFLOW_SERVER_URL'].rstrip('/')}/api/v1/run/{os.environ['LANGFLOW_FLOW_ID']}"

    headers = {
        "Content-Type": "application/json",
        "x-api-key": os.environ["LANGFLOW_API_KEY"],
    }

    payload = {
        "input_value": message,
        "session_id": session_id,
        "input_type": "chat",
        "output_type": "chat",
        "output_component": "",
        "tweaks": tweaks,
    }

    response = requests.post(url, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()


# EXAMPLE: replace "IBMwatsonxModel-XXXX" with your component ID from Langflow
example_tweaks = {
    "IBMwatsonxModel-XXXX": {
        "model_name": "ibm/granite-3-2b-instruct",  # example smaller model
        "temperature": 0.1,
    }
}

print("Example tweaks dict defined as `example_tweaks`.")

## 9. Next steps and workshop ideas

Now you have:

- A running Langflow instance configured to talk to IBM watsonx.ai
- A basic chat flow using the **IBM watsonx.ai** bundle component
- A Python helper to call that flow through the **Langflow API**
- An example of how to use **tweaks** to override model parameters at runtime

Ideas to extend this workshop:

- Build a **RAG flow** by adding IBM watsonx.ai Embeddings, a vector store, and
  document loaders.
- Turn your flow into a **tool** for watsonx Orchestrate (export JSON, then
  import as a tool).
- Add **agents** in Langflow (Agent components) and point them to your IBM
  watsonx.ai Language Model.
- Secure your Langflow server with authentication and a reverse proxy, following
  the Langflow best practices.

You can reuse this notebook as a starting point for internal trainings or
hackathons that focus on **IBM watsonx.ai + Langflow**.
